### Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#Sklearn
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

#other
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

# Plotly 
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

#spaCy
import spacy
nlp = spacy.load("fr_core_news_lg")
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
import string
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

### Load (acceptably clean) data

In [4]:
data=pd.read_pickle("../../data/cleaned2.pkl")
data_np=data.to_numpy()

In [5]:
i=random.randint(1, 40000)
data_np[i]

"stock des fabricants d'aliments biologiques pour bétail. FranceAgriMer - Établissement national des produits de l'agriculture et de la mer. grains graines en stock chez les fabricants d'aliments biologiques pour bétail des dernières campagnes. cereales oleagineux oleoproteagineux proteagineux tourteaux"

In [6]:
import progressbar
bar = progressbar.ProgressBar(maxval=len(data_np), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])


## Process data to extract keywords only

### Extract the locs 

In [7]:
def extract_locs(doc):
    all_locs=[None]*len(doc)
    bar.start()
    for i in range(len(doc)):
        this_doc=[]
        for token in nlp(doc[i]):
            if token.ent_type_=='LOC':
                this_doc.append(token.text)
        all_locs[i]=this_doc
        bar.update(i+1)
    bar.finish()
    return(all_locs)

import unicodedata
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [31]:
all_locs=extract_locs(data_np)

[========================================================================] 100%


In [93]:
stop_loc=['Région', 'Métropole', 'Region', 'Metropole','Mer', 'mer', 'Département', 'DEPARTEMENT', 'Agglomération', 'agglomération','Communauté', 'communauté']

In [94]:
all_locs_ns=[None]*len(all_locs)
for i in range(len(all_locs)):
    mini_list=[x for x in all_locs[i] if x not in stop_loc]
    all_locs_ns[i]=mini_list

In [95]:
all_locs_ns_str=all_locs_ns
for i in range(len(all_locs_ns)):
    all_locs_ns_str[i]=' '.join(all_locs_ns[i])
    

In [96]:
len(all_locs_ns_str)

42838

In [97]:
all_locs_np=np.array(all_locs)

In [98]:
np.save('../../data/locs_insee_str.npy', all_locs_ns_str, allow_pickle=True)

### Compare with insee tables

In [8]:
all_locs_ns_str=np.load('../../data/locs_insee_str.npy')

In [9]:
departements=pd.read_csv('../../data/departement2019.csv')
communes=pd.read_csv('../../data/communes-01012019.csv')
regions=pd.read_csv('../../data/region2019.csv')

In [10]:
communes_names=communes['nccenr'].to_numpy()

In [11]:
departements_names=departements['nccenr'].to_numpy()

In [12]:
regions_names=regions['nccenr'].to_numpy()

In [13]:
not_comm_names=list(departements_names)+list(regions_names)##+['France', 'Europe', 'Monde']

In [21]:
communes_prblm=[]
for i in communes_names:
    for j in not_comm_names:
        if fuzz.token_set_ratio(i,j)==100:
            communes_prblm.append(i)
    

In [22]:
all_locs_ns_str[25]

'île-de-france Île-de-France'

In [23]:
regions_names

array(['Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte',
       'Île-de-France', 'Centre-Val de Loire', 'Bourgogne-Franche-Comté',
       'Normandie', 'Hauts-de-France', 'Grand Est', 'Pays de la Loire',
       'Bretagne', 'Nouvelle-Aquitaine', 'Occitanie',
       'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur", 'Corse'],
      dtype=object)

In [49]:
from fuzzywuzzy import fuzz
def is_in(locs, category):
    values=[]
    for i in category:
        values.append(fuzz.token_set_ratio(locs, i))
        maxi=max(values)
    max_item=[i for i, j in enumerate(values) if j == maxi]   
   # print(max_item)
    if values[max_item[0]]==100:
        found=True
        if len(max_item)>1:
            values2=[]
            for w in max_item:
                values2.append(fuzz.ratio(locs, category[w]))
          #  print(values2)
            max_item_item=values2.index(max(values2))
            max_item=[max_item[max_item_item]]
    else:
        found=False
    return(max_item[0], found)

In [91]:
def text_to_loc(text):
    if text=='':
        return(None)
    if fuzz.token_set_ratio(text, 'france')==100:
        if text.find('Fort-de-France') == -1:
            return('France')
    max_item, found_c=is_in(text, communes_names)
    location=communes.loc[[max_item]]
    if communes_names[max_item] in communes_prblm:
        found_c=False
    if found_c==False:
        max_item, found_d=is_in(text, not_comm_names)
        try:
            location=departements.loc[[max_item]]
        except:
            location=regions.loc[[max_item-len(departements_names)]]
        return(location)
    return(location)

In [90]:
'Fort-de-France'.find('Fort-de-France')

0

In [105]:
i=random.randint(1,len(all_locs_ns_str))
#i=27944
print('TEXTE INITIAL: '+ data_np[i])
t = time.time()
location=text_to_loc(all_locs_ns_str[i])
elapsed = time.time() - t
#print('MOTS EXTRAITS'+ all_locs_ns_str[i])
print(' ')
print('Localisation INSEE:')
print(location)
#print('Computed in '+str(elapsed) )

TEXTE INITIAL: extrait de la base sirene sur le périmètre de la commune de nice. Métropole Nice Côte d'Azur. vous trouverez ici l'extrait de la base nationale sirene sur le périmètre de la commune de nice.. base entreprise nice sirene
 
Localisation INSEE:
     typecom    com   reg dep  arr  tncc   ncc nccenr libelle   can  comparent
2069     COM  06088  93.0  06  062     0  NICE   Nice    Nice  0699        NaN


In [45]:
all_locs_ns_str[20751]

'rivière marne de châlons-en-champagne Direction Départementale des Territoires de la Marne'

In [224]:
communes_names[31343]#8183

'Porte-de-Savoie'

In [122]:
other=list(regions_names)+list(departements_names)+['France']

In [44]:
communes

,typecom,com,reg,dep,arr,tncc,ncc,nccenr,libelle,can,comparent
0,COM,01001,84.0,01,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...
37925,COM,97613,6.0,976,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN
37926,COM,97614,6.0,976,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN
37927,COM,97615,6.0,976,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN
37928,COM,97616,6.0,976,NaN,0,SADA,Sada,Sada,97612,NaN


In [144]:
dpt_prblm=[]
for i in departements_names:
    for j in regions_names:
        if fuzz.token_set_ratio(i,j)==100:
            dpt_prblm.append(i)

In [48]:
for i in communes_names:
    if fuzz.token_set_ratio(i,'drôme')==100:
        print(i)

Balleroy-sur-Drôme
Lande-sur-Drôme
Val de Drôme
Livron-sur-Drôme
Loriol-sur-Drôme


In [146]:
regions_names

array(['Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte',
       'Île-de-France', 'Centre-Val de Loire', 'Bourgogne-Franche-Comté',
       'Normandie', 'Hauts-de-France', 'Grand Est', 'Pays de la Loire',
       'Bretagne', 'Nouvelle-Aquitaine', 'Occitanie',
       'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur", 'Corse'],
      dtype=object)

In [164]:
departements_names

array(['Ain', 'Aisne', 'Allier', 'Alpes-de-Haute-Provence',
       'Hautes-Alpes', 'Alpes-Maritimes', 'Ardèche', 'Ardennes', 'Ariège',
       'Aube', 'Aude', 'Aveyron', 'Bouches-du-Rhône', 'Calvados',
       'Cantal', 'Charente', 'Charente-Maritime', 'Cher', 'Corrèze',
       "Côte-d'Or", "Côtes-d'Armor", 'Creuse', 'Dordogne', 'Doubs',
       'Drôme', 'Eure', 'Eure-et-Loir', 'Finistère', 'Corse-du-Sud',
       'Haute-Corse', 'Gard', 'Haute-Garonne', 'Gers', 'Gironde',
       'Hérault', 'Ille-et-Vilaine', 'Indre', 'Indre-et-Loire', 'Isère',
       'Jura', 'Landes', 'Loir-et-Cher', 'Loire', 'Haute-Loire',
       'Loire-Atlantique', 'Loiret', 'Lot', 'Lot-et-Garonne', 'Lozère',
       'Maine-et-Loire', 'Manche', 'Marne', 'Haute-Marne', 'Mayenne',
       'Meurthe-et-Moselle', 'Meuse', 'Morbihan', 'Moselle', 'Nièvre',
       'Nord', 'Oise', 'Orne', 'Pas-de-Calais', 'Puy-de-Dôme',
       'Pyrénées-Atlantiques', 'Hautes-Pyrénées', 'Pyrénées-Orientales',
       'Bas-Rhin', 'Haut-Rhin', 'Rhône',

In [129]:
len(communes_prblm)

809

In [114]:
data_np[3422]

"concentrations en nitrates d'origine agricole - - eaux souterraines - france entière. Système d'Information sur l'Eau. la directive 91 cee du 12 décembre dite directive nitrates vise protéger la qualité de eau en prévenant la pollution des eaux souterraines et superficielles par les nitrates d’origine agricole notamment en promouvant usage des bonnes pratiques agricoles de gestion de l’azote. ses orientations sont largement reprises dans la directive cadre sur l’eau dce 60 ce du 23 octobre . elle impose aux états membres de : •\tréaliser des campagnes de surveillance des concentrations en nitrates dans les milieux aquatiques eaux superficielles et souterraines au moins tous les quatre ans •\tdésigner des zones de protection spécifiques atteintes ou menacées par la pollution par les nitrates d’origine agricole dites zones vulnérables •\télaborer des programmes d’actions dont les mesures doivent être obligatoirement appliquées par les agriculteurs qui exercent leurs activités en zones v